# LIBRARIES

In [1]:
cd ..

/Users/daverod/Downloads/optimizely_cs


In [2]:
import numpy as np
import pandas as pd
import itertools #to list the films
import sys
import pickle #to save/call the model
from ast import literal_eval #to extract text from dictionaries
sys.path.append("../../")

#hybrid model of choice
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.data import Dataset

#subdirectory from Microsoft GitHub devoted to recommenders - has useful metrics used here
from reco_utils.evaluation.python_evaluation import precision_at_k, recall_at_k

/Users/daverod/opt/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


# LOAD & PREP

In [3]:
def load_data(infile1, infile2):
    a = pd.read_csv(infile1)
    a.rename(columns={'userId': 'userID','movieId':'itemID'}, inplace=True)

    #chose genre as the text feature (item attribute)
    df = pd.read_csv(infile2)[['id','genres']]
    df2 = df.drop(df[df['genres'] == '[]'].index, axis=0).reset_index(drop=True)
    df2['genres'] = df2['genres'].apply(lambda x: [x['name'] for x in literal_eval(x)])
    df2.rename(columns={'id': 'itemID'}, inplace=True)

    movie_genre = df2['genres']
    all_movie_genre = sorted(list(set(itertools.chain.from_iterable(movie_genre))))

    #make a single base dataset
    new_data = a.join(df2, on='itemID', how='left', lsuffix='a')
    new_data.drop(['timestamp','itemID'],axis=1,inplace=True)
    new_data.rename(columns={'itemIDa': 'itemID'}, inplace=True)

    dataset = Dataset()
    dataset.fit(a['userID'], a['itemID'], item_features=all_movie_genre)
    item_features = dataset.build_item_features((x, y) for x,y in zip(a.itemID, movie_genre))

    (interactions, weights) = dataset.build_interactions(a.iloc[:, 0:3].values)

    return a, movie_genre, item_features, interactions, weights, all_movie_genre

a, movie_genre, item_features, interactions, weights, all_movie_genre = load_data(infile1='/Users/daverod/Downloads/optimizely_cs/data/ratings_small.csv'
,infile2='/Users/daverod/Downloads/optimizely_cs/data/movies_metadata_extract.csv')

# SPLIT

In [4]:
def split_data(ints):
    #scatter into train + test
    train_interactions, test_interactions = cross_validation.random_train_test_split(ints, test_percentage=0.25,
random_state=np.random.RandomState(42))
    
    return train_interactions, test_interactions

train_interactions, test_interactions = split_data(interactions)

# TRAIN

In [5]:
def model_data(trnint, atts):
    model = LightFM(loss='warp', no_components=25, learning_rate=0.25, 
                     item_alpha=1e-6, user_alpha=1e-6, random_state=np.random.RandomState(42))

    model.fit(interactions=trnint, item_features=atts, epochs=20)
    
    return model

model = model_data(train_interactions, item_features)

In [6]:
def pre_prep(ints,inp,lst1,lst2):
    uids, iids, interaction_data = cross_validation._shuffle(ints.row, ints.col
                            , ints.data, random_state=np.random.RandomState(42))
    
    cutoff = int((1.0 - 0.25) * len(uids))
    
    test_idx = slice(cutoff, None)
    
    dataset = Dataset()
    dataset.fit(a['userID'], a['itemID'], item_features=all_movie_genre)
    item_features = dataset.build_item_features((x, y) for x,y in zip(a.itemID, movie_genre))
    (interactions, weights) = dataset.build_interactions(a.iloc[:, 0:3].values)
    
    uid_map, ufeature_map, iid_map, ifeature_map = dataset.mapping()
    
    return test_idx, uids, iids, uid_map, iid_map

test_idx, uids, iids, uid_map, iid_map = pre_prep(interactions, a, all_movie_genre, movie_genre)

In [7]:
#get actual ratings for test set
def prepare_test_df(test_idx, uids, iids, uid_map, iid_map, weights):
    test_df = pd.DataFrame(
        zip(
            uids[test_idx],
            iids[test_idx],
            [list(uid_map.keys())[x] for x in uids[test_idx]],
            [list(iid_map.keys())[x] for x in iids[test_idx]],
        ),
        columns=["uid", "iid", "userID", "itemID"],
    )

    dok_weights = weights.todok()
    test_df["rating"] = test_df.apply(lambda x: dok_weights[x.uid, x.iid], axis=1)

    return test_df[["userID", "itemID", "rating"]]

test_df = prepare_test_df(test_idx, uids, iids, uid_map, iid_map, weights)

In [8]:
def get_predictions(data, uid_map, iid_map, interactions, model, num_threads
                   , user_features, item_features):
    
    #get predicted ratings for test set - note the longest part of code (approx 20-40 mins)
    
    #num_threads: note my PC only has 2 cores - suggest to set this to cores of your PC
    
    users, items = [], []
    
    item = list(data.itemID.unique())
    
    for user in data.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        
    all_predictions = pd.DataFrame(data={"userID": users, "itemID": items})
    all_predictions["uid"] = all_predictions.userID.map(uid_map)
    all_predictions["iid"] = all_predictions.itemID.map(iid_map)
    dok_weights = interactions.todok()
    all_predictions["rating"] = all_predictions.apply(lambda x: dok_weights[x.uid, x.iid], axis=1)
    all_predictions = all_predictions[all_predictions.rating < 1].reset_index(drop=True)
    all_predictions = all_predictions.drop("rating", axis=1)
    
    all_predictions['prediction'] = all_predictions.apply(lambda x: model.predict(user_ids=int(x["uid"]),
                                              item_ids=[x["iid"]],
            user_features=user_features,item_features=item_features,num_threads=num_threads,)[0], axis=1,)
    
    return all_predictions[['userID','itemID','prediction']]

all_predictions = get_predictions(data=a, uid_map=uid_map, iid_map=iid_map, interactions=train_interactions
                , model=model, num_threads=2, user_features=None, item_features=item_features)

# EVAL

In [9]:
#focus on precision (% actually correct) + recall (coverage of all films in predictions)
def eval_metrics(rating_true, rating_pred, k):
    precision = precision_at_k(rating_true=rating_true, rating_pred=rating_pred, k=k)

    recall = recall_at_k(rating_true=rating_true, rating_pred=rating_pred, k=k)

    print(f"Precision@K:\t{precision:.3f}", f"Recall@K:\t{recall:.3f}", sep='\n')
    
eval_metrics(rating_true=test_df, rating_pred=all_predictions, k=10)

Precision@K:	0.123
Recall@K:	0.047


- **RANK 20, TOP 10:**
*PRECISION: 0.093, RECALL: 0.035*
- **RANK 25, TOP 10:**
*PRECISION: 0.123, RECALL: 0.047*
- **RANK 30, TOP 10:**
*PRECISION: 0.096, RECALL: 0.037*
- **RANK 40, TOP 10:**
*PRECISION: 0.050, RECALL: 0.026*
---
- **RANK 20, TOP 20:**
*PRECISION: 0.103, RECALL: 0.078*
- **RANK 30, TOP 20:**
*PRECISION: 0.106, RECALL: 0.076*
- **RANK 40, TOP 20:**
*PRECISION: 0.059, RECALL: 0.052*
---
**<font color=Red>Best model for top 10 has rank of 25</font>**

# SAVE

In [10]:
def save_file(dset, to):
    return dset.to_csv(to ,index=False)

save_file(dset=all_predictions, to='/Users/daverod/Downloads/optimizely_cs/data/the_prediction_file.csv')

In [11]:
def save_model(mdl, to):
    return pickle.dump(mdl, open(to, 'wb'))

save_model(mdl=model, to='/Users/daverod/Downloads/optimizely_cs/model/the_model_file.sav')